In [29]:
# Импорт необходимых модулей
from imutils.video import VideoStream, FPS
import cv2
import numpy as np
import dlib
from tensorflow import keras

# загрузим предобученную модель распознавания эмоций по контрольным точкам
model = keras.models.load_model('')

# Запуск видео потока
vs = VideoStream(src=0).start()

# приводим метки к категориям
dict_genre = {'anger' : 1, 'contempt' : 2, 'disgust' : 3, 'fear' : 4, 'happy' : 5,
       'neutral' : 6, 'sad' : 7, 'surprise' : 8, 'uncertain' : 9}

# Подключение детектора, настроенного на поиск человеческих лиц
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
 
    # Получение изображения из видео потока
    frame = vs.read()

    # Конвертирование изображения в черно-белое
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Обнаружение лиц и построение прямоугольного контура
    faces = detector(grayFrame)

    # Обход списка всех лиц попавших на изображение
    for face in faces:

        # Выводим количество лиц на изображении
        #cv2.putText(frame, "{} face(s) found".format(len(faces)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Получение координат вершин прямоугольника и его построение на изображении
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)
 
        # Получение координат контрольных точек
        landmarks = predictor(grayFrame, face)
        l_points_face = []
        for n in [i for i in range(17, 27)]+[i for i in range(36, 68)]:
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            l_points_face.append(x)
            l_points_face.append(y)
            
        l_points_face = [i - l_points_face[0] for i in l_points_face] # начало координат в первую точку
        l_points_face = [i / frame.shape[0] for i in l_points_face] # нормализация
        
        # подготовка инпута для нейронной сети
        x = tf.constant(l_points_face, shape=[1, 84])
        
        # предсказание эмоции
        predict = model.predict_classes(x)
        
        # метку в читаемый вид
        pred_label = [name for name, cat in dict_genre.items() if cat == predict][0]
        # наложение метки на видео
        cv2.putText(frame,pred_label, (x1+100, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            

    cv2.putText(frame, "Press ESC to close frame", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Вывод преобразованного изображения
    cv2.imshow("Frame", frame)

    # Для выхода из цикла нажать ESC
    key = cv2.waitKey(1)
    if key == 27:
        break

# закрываем окно        
cv2.destroyAllWindows()